# ETL Project



In [4]:
import pandas as pd
import psycopg2   

import warnings
warnings.filterwarnings("ignore")


## We preprocessed the lending club dataset(that we got from data.world) to remove null values and columns that are not needed for modeling  and stored it in filtered_loans.csv
### Get data from filtered_loan.csv into one dataframe

In [5]:
loans_df = pd.read_csv("filtered_loans.csv")
#Convert member id to int, e.g member_id is 1296599.0 take only 1296599
loans_df['member_id'] = loans_df['member_id'].astype(int)
loans_df

,id,member_id,loan_amnt,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,...,initial_list_status,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,tax_liens,fico_average
0,1077501,1296599,5000.0,36 months,10.65%,162.87,B,10+ years,RENT,24000.0,...,f,Sep-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,737.0
1,1077430,1314167,2500.0,60 months,15.27%,59.83,C,< 1 year,RENT,30000.0,...,f,Sep-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,742.0
2,1077175,1313524,2400.0,36 months,15.96%,84.33,C,10+ years,RENT,12252.0,...,f,Sep-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,737.0
3,1076863,1277178,10000.0,36 months,13.49%,339.31,C,10+ years,RENT,49200.0,...,f,Apr-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,692.0
4,1075358,1311748,3000.0,60 months,12.69%,67.79,B,1 year,RENT,80000.0,...,f,Sep-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,697.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41216,122515,122503,1500.0,36 months,11.86%,49.72,D,5 years,RENT,28000.0,...,f,Oct-2010,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,667.0
41217,122065,122062,3000.0,36 months,8.38%,94.54,A,< 1 year,RENT,20000.0,...,f,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,732.0
41218,121673,121283,4500.0,36 months,8.07%,141.15,A,< 1 year,RENT,18240.0,...,f,Oct-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,737.0
41219,120371,120368,15000.0,36 months,12.17%,499.45,D,1 year,MORTGAGE,83200.0,...,f,Apr-2015,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,712.0


## Breaking the single dataframe into member_loan_details, member_credit_history, member_delinq for database insert

### Member loans dataframe

In [6]:
#member loan details
member_loan_df  = loans_df[['id','member_id','emp_length','home_ownership','annual_inc','addr_state'
                                  ,'loan_amnt','term','int_rate','installment','purpose','grade','verification_status','initial_list_status']]

member_loan_df['home_ownership'] = member_loan_df['home_ownership'].str.title()
member_loan_df

,id,member_id,emp_length,home_ownership,annual_inc,addr_state,loan_amnt,term,int_rate,installment,purpose,grade,verification_status,initial_list_status
0,1077501,1296599,10+ years,Rent,24000.0,AZ,5000.0,36 months,10.65%,162.87,credit_card,B,Verified,f
1,1077430,1314167,< 1 year,Rent,30000.0,GA,2500.0,60 months,15.27%,59.83,car,C,Source Verified,f
2,1077175,1313524,10+ years,Rent,12252.0,IL,2400.0,36 months,15.96%,84.33,small_business,C,Not Verified,f
3,1076863,1277178,10+ years,Rent,49200.0,CA,10000.0,36 months,13.49%,339.31,other,C,Source Verified,f
4,1075358,1311748,1 year,Rent,80000.0,OR,3000.0,60 months,12.69%,67.79,other,B,Source Verified,f
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41216,122515,122503,5 years,Rent,28000.0,FL,1500.0,36 months,11.86%,49.72,other,D,Not Verified,f
41217,122065,122062,< 1 year,Rent,20000.0,NY,3000.0,36 months,8.38%,94.54,educational,A,Not Verified,f
41218,121673,121283,< 1 year,Rent,18240.0,GA,4500.0,36 months,8.07%,141.15,other,A,Not Verified,f
41219,120371,120368,1 year,Mortgage,83200.0,WI,15000.0,36 months,12.17%,499.45,credit_card,D,Not Verified,f


### member credit history dataframe

In [7]:
# Member credit history
member_credit_hist_df = loans_df[['id','member_id','earliest_cr_line','last_credit_pull_d','inq_last_6mths',
                             'open_acc','pub_rec','revol_util','total_acc','fico_average','dti','revol_bal']]
member_credit_hist_df

,id,member_id,earliest_cr_line,last_credit_pull_d,inq_last_6mths,open_acc,pub_rec,revol_util,total_acc,fico_average,dti,revol_bal
0,1077501,1296599,Jan-1985,Sep-2016,1.0,3.0,0.0,83.7%,9.0,737.0,27.65,13648.0
1,1077430,1314167,Apr-1999,Sep-2016,5.0,3.0,0.0,9.4%,4.0,742.0,1.00,1687.0
2,1077175,1313524,Nov-2001,Sep-2016,2.0,2.0,0.0,98.5%,10.0,737.0,8.72,2956.0
3,1076863,1277178,Feb-1996,Apr-2016,1.0,10.0,0.0,21%,37.0,692.0,20.00,5598.0
4,1075358,1311748,Jan-1996,Sep-2016,0.0,15.0,0.0,53.9%,38.0,697.0,17.94,27783.0
...,...,...,...,...,...,...,...,...,...,...,...,...
41216,122515,122503,Feb-2006,Oct-2010,1.0,1.0,0.0,0%,2.0,667.0,14.31,0.0
41217,122065,122062,Dec-1998,Jun-2016,9.0,4.0,0.0,27.4%,4.0,732.0,6.72,7021.0
41218,121673,121283,Apr-2004,Oct-2013,1.0,1.0,0.0,0%,2.0,737.0,3.29,0.0
41219,120371,120368,Oct-1995,Apr-2015,5.0,14.0,0.0,59.5%,37.0,712.0,17.02,37570.0


### member delinq dataframe

In [8]:
member_delinq_df = loans_df[['id','member_id','delinq_2yrs','collections_12_mths_ex_med','acc_now_delinq',
                             'chargeoff_within_12_mths','tax_liens']]
member_delinq_df

,id,member_id,delinq_2yrs,collections_12_mths_ex_med,acc_now_delinq,chargeoff_within_12_mths,tax_liens
0,1077501,1296599,0.0,0.0,0.0,0.0,0.0
1,1077430,1314167,0.0,0.0,0.0,0.0,0.0
2,1077175,1313524,0.0,0.0,0.0,0.0,0.0
3,1076863,1277178,0.0,0.0,0.0,0.0,0.0
4,1075358,1311748,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
41216,122515,122503,1.0,0.0,0.0,0.0,0.0
41217,122065,122062,0.0,0.0,0.0,0.0,0.0
41218,121673,121283,0.0,0.0,0.0,0.0,0.0
41219,120371,120368,0.0,0.0,0.0,0.0,0.0


## Drop tables function

In [9]:
#Drop table script for dropping each table, if exists in leding_club_loans database
def drop_tables(cursor):
    """ drop tables in the PostgreSQL database"""
    drop_table_commands =  (
        """
        DROP TABLE IF EXISTS member_credit_hist
        """,
        """
        DROP TABLE IF EXISTS member_delinq
        """,
        """ 
        DROP TABLE IF EXISTS member_loan_dtls
        """)
    # drop table one by one
    for drop_table_query in drop_table_commands:
        cursor.execute(drop_table_query)
        

## Create tables function

In [10]:
#Create table script for creating all the tables in FDA_food_event database
def create_tables(cursor):
    
    
    """ create tables in the PostgreSQL database"""
    create_table_commands =  (
        """
        CREATE TABLE member_loan_dtls
        ( 
             id INT PRIMARY KEY,
             member_id INT,
             emp_length VARCHAR(25),
             home_ownership VARCHAR(25),
             annual_inc NUMERIC,
             addr_state CHAR(2),
             loan_amnt NUMERIC,
             term CHAR(15),
             int_rate VARCHAR(10),
             installment NUMERIC,
             purpose VARCHAR(100),
             grade CHAR(1),
             verification_status VARCHAR(20) ,
             initial_list_status CHAR(1)
         )
        """,
        """
        CREATE TABLE member_credit_hist
         (
             id INT NOT NULL,
             member_id INT NOT NULL,
             earliest_cr_line VARCHAR(25),
             last_credit_pull_d VARCHAR(25),
             inq_last_6mths NUMERIC,
             open_acc FLOAT,
             pub_rec FLOAT,
             revol_util VARCHAR(10),
             total_acc FLOAT,
             fico_average FLOAT,
             dti FLOAT,
             revol_bal NUMERIC,
             FOREIGN KEY(id) REFERENCES member_loan_dtls(id)
         )
        """,
        """
        CREATE TABLE member_delinq
         (   id integer NOT NULL,
             member_id integer NOT NULL,
             delinq_2yrs decimal, 
             collections_12_mths_ex_med decimal,
             acc_now_delinq decimal,
             chargeoff_within_12_mths decimal,
             tax_liens decimal,
             FOREIGN KEY(id) REFERENCES member_loan_dtls(id)
         );
        """)
     # create table one by one
    for create_table_query in create_table_commands:
        cursor.execute(create_table_query)
        

## Function to insert into tables using copy_from() file

In [11]:
#Copy data from .csv file to table in FDA_food_event database. Using '^' as separator for .csv file.
def copy_data_from_file(cursor, df, table):
    """
    Here we are going save the dataframe on disk as 
    a csv file, load the csv file and use copy_from() to copy it to the table
    """
    
    # Save the dataframe to disk
    tmp_df = "./tmp_" + table + ".csv"
    df.to_csv(tmp_df, index=False, header=False,sep='^')
    f = open(tmp_df, 'r')
    cursor.copy_from(f, table, sep="^",null='')
    

## Function to load lending_club_loans_databasedatabase
- database.ini file has connection parameters under section postgresql

    #### For Azure Postgres Database server
    Azure Postgres server is created in Azure portal with name \<servername\> and user name as \<username\>

    [postgresql] <br>
    host=\<servername\>.postgres.database.azure.com <br>
    database=lending_club_loans <br>
    port = 5432 <br>
    user=\<username\>@\<servername\> <br>
    password=password <br>

    #### For local Postgres Database server
    [postgresql] <br>
    host=localhost <br>
    database=lending_club_loans <br>
    user=postgres <br>
    password=password <br>

    
- Config.py has Config function which parses through the postgresql section of database.ini file to get connection paramaters. Below 2 lines are code to do that.
    - params = config()
    - conn = psycopg2.connect(**params)

In [12]:
#to get database connection parameters like host,database,user and password
from config import config
def load_lending_club_loans_database(): 

    conn = None
    try:
        # read the connection parameters from database.ini and config
        params = config()
        
        # connect to the PostgreSQL server, using the parameters host, database,user and password and 
        #port(5432 - in case of connecting to azure postgres DB)
        conn = psycopg2.connect(**params)
        cursor = conn.cursor()
        
        #Set the client_min_messages paramter to 'ERROR' to avoid reporting NOTICE
        set_command = (""" SET client_min_messages='ERROR' """)
        cursor.execute(set_command)
        
                     
        #Drop tables one by one
        drop_tables(cursor)
        
        #Create table one by one
        create_tables(cursor)
        
       
        #Insert into member_loan_dtls table
        copy_data_from_file(cursor, member_loan_df, 'member_loan_dtls')
        #Insert into member_credit_hist table
        copy_data_from_file(cursor, member_credit_hist_df, 'member_credit_hist')
        #Insert into member_delinq table
        copy_data_from_file(cursor, member_delinq_df, 'member_delinq')
             
        # close communication with the PostgreSQL database server
        cursor.close()
        # commit the changes
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        if conn is not None:   
            conn.rollback()
        
    finally:
        if conn is not None:
            conn.close()
            cursor.close()


## Call function to load lending_club_loans_database

In [13]:
#Load FDA_food_event database
load_lending_club_loans_database()

## Select data from tables

In [14]:
params = config()
with psycopg2.connect(**params) as conn:
    #Get member loan details
    m_loan_df = pd.read_sql_query("select * from member_loan_dtls;", conn)
    
    #Get member credit history
    m_credit_hist = pd.read_sql_query("select * from member_credit_hist;", conn)
    
    #Get member delinq details
    m_delinq = pd.read_sql_query("select * from member_delinq;", conn)
    
    #Get member loan, credit and delinq details using SQL join
    sqlJoinQuery = """ SELECT ml.*
        ,earliest_cr_line,last_credit_pull_d,inq_last_6mths,open_acc,pub_rec,revol_util,total_acc,fico_average,dti,revol_bal
        ,delinq_2yrs,collections_12_mths_ex_med,acc_now_delinq,chargeoff_within_12_mths,tax_liens 
        FROM member_loan_dtls ml
        INNER JOIN member_credit_hist mc
        ON ml.id = mc.id and ml.member_id  = mc.member_id
        INNER JOIN member_delinq md
        ON ml.id = md.id and ml.member_id  = md.member_id
                    """
    lending_club_loan_df = pd.read_sql_query(sqlJoinQuery, conn)

In [15]:
m_loan_df

,id,member_id,emp_length,home_ownership,annual_inc,addr_state,loan_amnt,term,int_rate,installment,purpose,grade,verification_status,initial_list_status
0,1077501,1296599,10+ years,Rent,24000.0,AZ,5000.0,36 months,10.65%,162.87,credit_card,B,Verified,f
1,1077430,1314167,< 1 year,Rent,30000.0,GA,2500.0,60 months,15.27%,59.83,car,C,Source Verified,f
2,1077175,1313524,10+ years,Rent,12252.0,IL,2400.0,36 months,15.96%,84.33,small_business,C,Not Verified,f
3,1076863,1277178,10+ years,Rent,49200.0,CA,10000.0,36 months,13.49%,339.31,other,C,Source Verified,f
4,1075358,1311748,1 year,Rent,80000.0,OR,3000.0,60 months,12.69%,67.79,other,B,Source Verified,f
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41216,122515,122503,5 years,Rent,28000.0,FL,1500.0,36 months,11.86%,49.72,other,D,Not Verified,f
41217,122065,122062,< 1 year,Rent,20000.0,NY,3000.0,36 months,8.38%,94.54,educational,A,Not Verified,f
41218,121673,121283,< 1 year,Rent,18240.0,GA,4500.0,36 months,8.07%,141.15,other,A,Not Verified,f
41219,120371,120368,1 year,Mortgage,83200.0,WI,15000.0,36 months,12.17%,499.45,credit_card,D,Not Verified,f


In [16]:
m_credit_hist

,id,member_id,earliest_cr_line,last_credit_pull_d,inq_last_6mths,open_acc,pub_rec,revol_util,total_acc,fico_average,dti,revol_bal
0,1077501,1296599,Jan-1985,Sep-2016,1.0,3.0,0.0,83.7%,9.0,737.0,27.65,13648.0
1,1077430,1314167,Apr-1999,Sep-2016,5.0,3.0,0.0,9.4%,4.0,742.0,1.00,1687.0
2,1077175,1313524,Nov-2001,Sep-2016,2.0,2.0,0.0,98.5%,10.0,737.0,8.72,2956.0
3,1076863,1277178,Feb-1996,Apr-2016,1.0,10.0,0.0,21%,37.0,692.0,20.00,5598.0
4,1075358,1311748,Jan-1996,Sep-2016,0.0,15.0,0.0,53.9%,38.0,697.0,17.94,27783.0
...,...,...,...,...,...,...,...,...,...,...,...,...
41216,122515,122503,Feb-2006,Oct-2010,1.0,1.0,0.0,0%,2.0,667.0,14.31,0.0
41217,122065,122062,Dec-1998,Jun-2016,9.0,4.0,0.0,27.4%,4.0,732.0,6.72,7021.0
41218,121673,121283,Apr-2004,Oct-2013,1.0,1.0,0.0,0%,2.0,737.0,3.29,0.0
41219,120371,120368,Oct-1995,Apr-2015,5.0,14.0,0.0,59.5%,37.0,712.0,17.02,37570.0


In [17]:
m_delinq

,id,member_id,delinq_2yrs,collections_12_mths_ex_med,acc_now_delinq,chargeoff_within_12_mths,tax_liens
0,1077501,1296599,0.0,0.0,0.0,0.0,0.0
1,1077430,1314167,0.0,0.0,0.0,0.0,0.0
2,1077175,1313524,0.0,0.0,0.0,0.0,0.0
3,1076863,1277178,0.0,0.0,0.0,0.0,0.0
4,1075358,1311748,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
41216,122515,122503,1.0,0.0,0.0,0.0,0.0
41217,122065,122062,0.0,0.0,0.0,0.0,0.0
41218,121673,121283,0.0,0.0,0.0,0.0,0.0
41219,120371,120368,0.0,0.0,0.0,0.0,0.0


In [18]:
lending_club_loan_df

,id,member_id,emp_length,home_ownership,annual_inc,addr_state,loan_amnt,term,int_rate,installment,...,revol_util,total_acc,fico_average,dti,revol_bal,delinq_2yrs,collections_12_mths_ex_med,acc_now_delinq,chargeoff_within_12_mths,tax_liens
0,1077501,1296599,10+ years,Rent,24000.0,AZ,5000.0,36 months,10.65%,162.87,...,83.7%,9.0,737.0,27.65,13648.0,0.0,0.0,0.0,0.0,0.0
1,1077430,1314167,< 1 year,Rent,30000.0,GA,2500.0,60 months,15.27%,59.83,...,9.4%,4.0,742.0,1.00,1687.0,0.0,0.0,0.0,0.0,0.0
2,1077175,1313524,10+ years,Rent,12252.0,IL,2400.0,36 months,15.96%,84.33,...,98.5%,10.0,737.0,8.72,2956.0,0.0,0.0,0.0,0.0,0.0
3,1075269,1311441,3 years,Rent,36000.0,AZ,5000.0,36 months,7.90%,156.46,...,28.3%,12.0,732.0,11.20,7963.0,0.0,0.0,0.0,0.0,0.0
4,1071795,1306957,4 years,Own,40000.0,CA,5600.0,60 months,21.28%,152.39,...,32.6%,13.0,677.0,5.55,5210.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41216,123855,123812,1 year,Rent,10192.0,IN,1800.0,36 months,14.07%,61.58,...,97.5%,4.0,637.0,5.06,2925.0,0.0,0.0,0.0,0.0,0.0
41217,123299,123295,1 year,Rent,32500.0,NY,11625.0,36 months,15.01%,403.07,...,63.5%,2.0,647.0,0.74,381.0,0.0,0.0,0.0,0.0,0.0
41218,123133,114358,3 years,Rent,27716.0,AZ,11050.0,36 months,15.96%,388.28,...,51.5%,15.0,647.0,12.90,2621.0,0.0,0.0,0.0,0.0,0.0
41219,122913,121255,1 year,Mortgage,22000.0,NH,6000.0,36 months,14.70%,207.11,...,36.2%,17.0,667.0,20.00,15782.0,0.0,0.0,0.0,0.0,0.0


In [19]:
## Create Database 

In [20]:
# Create database statement
#sqlCreateDatabase = "CREATE DATABASE lending_club_loans;"

# Create a table in PostgreSQL database
#cursor.execute(sqlCreateDatabase);